In [2]:
import xarray as xr
import pandas as pd
import numpy as np
import stackstac
import planetary_computer as pc
from dask.distributed import Client, LocalCluster
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")

# Import your modular function
from env_function import environmental_variables

# Start Dask (safe + required)
cluster = LocalCluster(
    n_workers=4,
    threads_per_worker=2,
    memory_limit="8GB"
)
client = Client(cluster)

client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 29.80 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35289,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:41773,Total threads: 2
Dashboard: http://127.0.0.1:37329/status,Memory: 7.45 GiB
Nanny: tcp://127.0.0.1:37283,


In [3]:
bbox = (-82.7167, 27.5833, -82.3833, 28.0333)  # Tampa Bay
start_date = "2019-01-01"
end_date = "2024-12-31"

env_data = environmental_variables(
    bbox=bbox,
    start_date=start_date,
    end_date=end_date,
    variables=["sst", "precip"]
)

sst_lazy = env_data["sst"]          # xarray DataArray (lazy)
precip_items = env_data["precip"]   # list of STAC Items

type(sst_lazy), type(precip_items)

SST data from s3://surftemp-sst/data/sst.zarr is truncated (stops at 2020-12-31T12:00:00.000000000). Trying fallback...
SST data successfully fetched from: s3://mur-sst/zarr-v1
Applying Kelvin to Celsius conversion.
Year 2019: 0 precip items from noaa-mrms-qpe-24h-pass2
Year 2020: 278 precip items from noaa-mrms-qpe-24h-pass2
Year 2021: 1988 precip items from noaa-mrms-qpe-24h-pass2
Year 2022: 3841 precip items from noaa-mrms-qpe-24h-pass2
Year 2023: 8737 precip items from noaa-mrms-qpe-24h-pass2
Year 2024: 8354 precip items from noaa-mrms-qpe-24h-pass2


(xarray.core.dataarray.DataArray, list)

In [4]:
# Refresh SAS tokens (SIGN EACH ITEM — REQUIRED)
for item in precip_items:
    pc.sign_inplace(item)

# Uniform subsample across entire time range
max_items = 800
indices = np.linspace(
    0, len(precip_items) - 1,
    max_items,
    dtype=int
)
precip_sample = [precip_items[i] for i in indices]

len(precip_sample)

800

In [8]:
precip_stack = stackstac.stack(
    precip_sample,
    assets=["cog"],
    epsg=4326,
    fill_value=np.nan,
)

precip_monthly = (
    precip_stack
    .mean(dim=["x", "y"])
    .squeeze(drop=True)
    .resample(time="1ME")
    .sum(min_count=1)
)


In [ ]:
final_ds = xr.Dataset({
    "sst": sst_lazy,
    "precip": precip_monthly
})

# Compute only the final result (safe)
final_df = final_ds.compute().to_dataframe()

final_df.head()

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(final_df.index, final_df["sst"], label="Monthly SST (°C)")
plt.plot(final_df.index, final_df["precip"], label="Monthly Precip (mm)")
plt.title("Monthly SST & Precipitation (2019–2024)")
plt.xlabel("Time")
plt.ylabel("Value")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# --- 1. Separate DataFrames ---

# Select the SST column and save it to a new DataFrame
sst_df = final_df[['sst']].copy()

# Select the Precipitation column and save it to a new DataFrame
precip_df = final_df[['precip']].copy()

# --- 2. Define File Paths ---
sst_path = "kalu_sst_data.csv"
precip_path = "kalu_precip_data.csv"

# --- 3. Save to CSV ---
sst_df.to_csv(sst_path, index=True, index_label='time')
print(f"✔ Successfully saved SST data to: {sst_path}")

precip_df.to_csv(precip_path, index=True, index_label='time')
print(f"✔ Successfully saved Precipitation data to: {precip_path}")

# --- 4. Cleanup Dask Resources (Crucial) ---
try:
    client.close()
    cluster.close()
    print("\nDask client and cluster successfully shut down.")
except Exception as e:
    print(f"Could not shut down Dask resources: {e}")

# Display the paths

In [ ]:
# cell one 
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt

from env_function import environmental_variables
import analysis  # your new module

plt.style.use("seaborn-v0_8")
plt.rcParams["figure.figsize"] = (10, 4)

bbox = (-83.0, 27.3, -82.4, 28.0)
start = "2019-01-01"
end   = "2024-12-31"

env = environmental_variables(bbox, start, end, ["sst", "precip"])
sst = env["sst"]
precip = env["precip"]

# from Grace's code:
# ndwi_daily, ndci_daily, ndti_daily already computed for same bbox and dates


In [ ]:
# Cell 2: monthly and seasonal means using analysis.py

sst_aggr    = analysis.compute_monthly_seasonal_means(sst)
precip_aggr = analysis.compute_monthly_seasonal_means(precip)
ndwi_aggr   = analysis.compute_monthly_seasonal_means(ndwi_daily)
ndci_aggr   = analysis.compute_monthly_seasonal_means(ndci_daily)
ndti_aggr   = analysis.compute_monthly_seasonal_means(ndti_daily)

sst_monthly    = sst_aggr["monthly"]
precip_monthly = precip_aggr["monthly"]
ndwi_monthly   = ndwi_aggr["monthly"]
ndci_monthly   = ndci_aggr["monthly"]
ndti_monthly   = ndti_aggr["monthly"]

sst_season    = sst_aggr["seasonal"]
precip_season = precip_aggr["seasonal"]
ndwi_season   = ndwi_aggr["seasonal"]
ndci_season   = ndci_aggr["seasonal"]
ndti_season   = ndti_aggr["seasonal"]


In [ ]:
# Cell 3: time-series and seasonal-cycle plots

analysis.plot_time_series(sst_monthly,    "SST (°C)")
analysis.plot_time_series(precip_monthly, "Precipitation (mm/day)")
analysis.plot_time_series(ndci_monthly,   "NDCI")

analysis.plot_seasonal_cycle(ndci_daily,  "NDCI")
analysis.plot_seasonal_cycle(precip,      "Precipitation")


In [ ]:
# Cell 4: build monthly DataFrames and compute correlation matrix

wqi_df = xr.Dataset({
    "NDWI": ndwi_monthly,
    "NDCI": ndci_monthly,
    "NDTI": ndti_monthly,
}).to_dataframe()

env_df = xr.Dataset({
    "SST":   sst_monthly,
    "PRECT": precip_monthly,
}).to_dataframe()

corr_matrix = analysis.compute_correlation_matrix(wqi_df, env_df)
corr_matrix


In [ ]:
# Cell 5: RMSE examples

rmse_ndci_sst   = analysis.compute_rmse(ndci_monthly.values, sst_monthly.values)
rmse_ndci_prect = analysis.compute_rmse(ndci_monthly.values, precip_monthly.values)
rmse_ndti_sst   = analysis.compute_rmse(ndti_monthly.values, sst_monthly.values)
rmse_ndti_prect = analysis.compute_rmse(ndti_monthly.values, precip_monthly.values)

print("RMSE NDCI–SST:   ", rmse_ndci_sst)
print("RMSE NDCI–PRECT: ", rmse_ndci_prect)
print("RMSE NDTI–SST:   ", rmse_ndti_sst)
print("RMSE NDTI–PRECT: ", rmse_ndti_prect)


In [ ]:
# Cell 6: PCA on combined monthly indices + environmental variables

pca_results = analysis.compute_pca(wqi_df, env_df, n_components=2)
loadings = pca_results["loadings"]
loadings


In [ ]:
# Cell 6b: visualize PCA loadings

analysis.plot_pca_results(loadings)
